### End-to-End Data Pipeline and Dashboard Creation for E-Commerce Analysis

#### Pipeline Implementation by  `Yitbarek Tesfaye`

##### Load the Dataset

In [4]:
import pandas as pd

# Load the dataset

df = pd.read_excel("Online Retail.xlsx")

# Preview data
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


##### Installing appropriate packeges

In [10]:
pip install pymysql sqlalchemy


   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   --------------------------- ------------ 30.7/45.0 kB ? eta -:--:--
   ------------------------------------ --- 41.0/45.0 kB 217.9 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 217.9 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 217.9 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 147.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


##### Clean and Preprocess

In [8]:
# Drop rows with missing CustomerID
df = df.dropna(subset=['CustomerID'])

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Remove duplicates
df = df.drop_duplicates()

#Remove outliers (e.g., negative Quantity or UnitPrice)
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

# Add Total Revenue column
df['TotalRevenue'] = df['Quantity'] * df['UnitPrice']
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalRevenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


 ##### Load into a Relational Database

In [11]:
import pandas as pd
from sqlalchemy import create_engine

# MySQL connection details
username = "root"  # Replace with your MySQL username
password = "root"  # Replace with your MySQL password
host = "localhost"  # MySQL server address
database = "ecommerce"  # Database name

# Create an SQLAlchemy engine
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")

# Load cleaned data into MySQL
df.to_sql("ecommerce_data", engine, if_exists="replace", index=False)

print("Data uploaded to MySQL successfully!")


Data uploaded to MySQL successfully!


##### Save to a CSV for Later Use

In [12]:
df.to_csv("cleaned_ecommerce_data.csv", index=False)

 ##### Add Segments, AOV, and Hourly Transactions

In [4]:
import pandas as pd

# Load the cleaned data
data = pd.read_csv('cleaned_ecommerce_data.csv')

# Step 1: Calculate Total Revenue for Each Row
data['TotalRevenue'] = data['Quantity'] * data['UnitPrice']

# Step 2: Add Customer Segments
# Aggregate Total Revenue per Customer
customer_revenue = data.groupby('CustomerID')['TotalRevenue'].sum().reset_index()

# Define segments
def assign_segment(revenue):
    if revenue > 1000:
        return 'High Spender'
    elif revenue > 500:
        return 'Medium Spender'
    else:
        return 'Low Spender'

# Apply the segmentation
customer_revenue['CustomerSegment'] = customer_revenue['TotalRevenue'].apply(assign_segment)

# Merge the segments back into the main data
data = data.merge(customer_revenue[['CustomerID', 'CustomerSegment']], on='CustomerID', how='left')

# Step 3: Calculate AOV (Average Order Value)
# Total Revenue and Count of Orders per Invoice
order_data = data.groupby('InvoiceNo').agg(
    TotalRevenuePerOrder=('TotalRevenue', 'sum'),
    TotalItems=('InvoiceNo', 'size')
).reset_index()

# Merge AOV-related fields back into the main dataset
data = data.merge(order_data[['InvoiceNo', 'TotalRevenuePerOrder']], on='InvoiceNo', how='left')

# Step 4: Add Hour of Transaction
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])  # Ensure datetime format
data['HourOfDay'] = data['InvoiceDate'].dt.hour

# Step 5: Save the Merged Data
data.to_csv('mergeddata.csv', index=False)

print("Merged data saved as 'mergeddata.csv'")


Merged data saved as 'mergeddata.csv'


##### Calculate Average Order Value (AOV)

In [2]:
import pandas as pd

# Load the cleaned data
data = pd.read_csv('cleaned_ecommerce_data.csv')

# Calculate Total Revenue per Invoice
data['TotalRevenue'] = data['Quantity'] * data['UnitPrice']

# Group by InvoiceNo to calculate the Total Revenue per Order
order_data = data.groupby('InvoiceNo').agg(
    TotalRevenue=('TotalRevenue', 'sum'),
    TotalOrders=('InvoiceNo', 'count')
).reset_index()

# Calculate AOV
aov = order_data['TotalRevenue'].sum() / order_data['InvoiceNo'].nunique()
print(f"Average Order Value (AOV): {aov}")



Average Order Value (AOV): 479.56016047917126


#####  Transactions by Hour of Day

In [3]:
# Ensure InvoiceDate is in datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

# Extract hour from InvoiceDate
data['HourOfDay'] = data['InvoiceDate'].dt.hour

# Count transactions by Hour of Day
hourly_transactions = data.groupby('HourOfDay').agg(
    Transactions=('InvoiceNo', 'nunique')  # Count distinct invoices
).reset_index()


print(hourly_transactions)


    HourOfDay  Transactions
0           6             1
1           7            29
2           8           555
3           9          1393
4          10          2226
5          11          2277
6          12          3130
7          13          2636
8          14          2274
9          15          2037
10         16          1100
11         17           544
12         18           169
13         19           144
14         20            18
